
<p style="text-align: center;font-size: 18px;">
    BIO/MIC 494/598 <span style="font-weight: bold;">Microbiome Data Science</span>
</p>
<p style="text-align: center;"><span style="font-size: 32px; font-weight: bold; background-color: #ffc627;">
    Sequencing Data Processing
</span></p>
<p style="text-align: center; font-size: 18px;">
    Team 5
</p>

## Data files

In [22]:
! git clone https://github.com/salorain/10251.git   

Cloning into '10251'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [31]:
!cd 10251

## From Qiita to Qiime2

In [3]:
!qiime tools import \
  --input-path 58889_reference-hit.biom \
  --type 'FeatureTable[Frequency]' \
  --output-path 10251table.qza


Imported 58889_reference-hit.biom as BIOMV210DirFmt to 10251table.qza


In [4]:
!qiime tools import \
  --input-path 58889_reference-hit.seqs.fa \
  --type 'FeatureData[Sequence]' \
  --output-path 10251rep-seqs.qza


Imported 58889_reference-hit.seqs.fa as DNASequencesDirectoryFormat to 10251rep-seqs.qza


!qiime tools import \
  --input-path 58889_insertion_tree.relabelled.tre \
  --type 'Phylogeny[Rooted]' \
  --output-path 10251rooted-tree.qza

In [5]:
!qiime feature-table summarize \
  --i-table 10251table.qza \
  --o-visualization 10251table.qzv \
  --m-sample-metadata-file 10251_prep_690_20201201-0-80233m.txt
!qiime feature-table tabulate-seqs \
  --i-data 10251rep-seqs.qza \
  --o-visualization 10251rep-seqs.qzv

Saved Visualization to: 10251table.qzv
Saved Visualization to: 10251rep-seqs.qzv


In [1]:
#Frequency of detection 
#from biom import load_table table = load_table('input.biom') 
#from biom.util import biom_open with biom_open('output.biom', 'w') as f: 
#table.to_hdf5(f, table.generated_by) 

In [6]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences 10251rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree 10251unrooted-tree.qza \
  --o-rooted-tree 102511rooted-tree.qza

Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: 10251unrooted-tree.qza
Saved Phylogeny[Rooted] to: 102511rooted-tree.qza


## Taxonomic analysis

In [7]:
!curl -sL \
  "https://data.qiime2.org/2021.2/common/gg-13-8-99-515-806-nb-classifier.qza" > \
  "gg-13-8-99-515-806-nb-classifier.qza"

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads 10251rep-seqs.qza \
  --o-classification taxonomy.qza

!qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

!qiime feature-table group \
  --i-table 10251table.qza \
  --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --m-metadata-column "experiment_title" \
--p-axis sample \
--p-mode sum \
--o-grouped-table grouped-table.qza

!qiime feature-table summarize \
  --i-table grouped-table.qza \
  --o-visualization grouped-table.qzv \
  --m-sample-metadata-file 10251/10251_prep_690_20201201-080233.txt


### Barplot

In [ ]:
!qiime taxa barplot \
  --i-table 10251table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file 10251_prep_690_20201201-0-80233m.txt \
  --o-visualization barplot.qzv

In [4]:
from qiime2.plugins.taxa.methods import collapse

In [5]:
ranks = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']

In [6]:
taxtables = {}
for i, rank in enumerate(ranks):
    table_ = collapse(table, taxonomy, i + 1).collapsed_table
    count = table_.view(pd.DataFrame).shape[1]
    taxtables[rank] = table_
    print(f'L{i + 1} ({rank}): {count}')

NameError: name 'table' is not defined

In [ ]:
taxtables['genus'].view(pd.DataFrame).T.head(10)

## ANCOM

In [ ]:
from qiime2.plugins.composition.methods import add_pseudocount

In [ ]:
table1 = add_pseudocount(table, 1).composition_table

In [ ]:
taxtable1s = {k: add_pseudocount(v, 1).composition_table for k, v in taxtables.items()}

In [ ]:
from qiime2.plugins.composition.visualizers import ancom

In [ ]:
resultAncom = ancom(taxtable1s['phylum'], metadata.get_column('experiment_title'))

In [ ]:
resultAncom.visualization

# Rarefaction

In [ ]:
!qiime feature-table rarefy \
  --i-table 10251table.qza \
  --p-sampling-depth 20102 \
  --o-rarefied-table rarefied_table.qza

Retained 5,849,682 (26.52%) features in 291 (98.64%) samples at the specifed sampling depth (20102).

In [ ]:
!qiime feature-table summarize \
  --i-table rarefied_table.qza \
  --o-visualization rarefied_table.qzv \
  --m-sample-metadata-file 10251/10251_prep_690_20201201-080233.txt

#Differential abundance analysis with gneiss



##Building linear models using balances


In [ ]:
!curl -sL "https://data.qiime2.org/2018.2/tutorials/gneiss/taxa.qza" > "taxa.qza"


In [ ]:
!qiime gneiss add-pseudocount \
  --i-table 10251table.qza \
  --p-pseudocount 1 \
  --o-composition-table composition.qza

### Alpha and beta diversity analysis

In [ ]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny 102511rooted-tree.qza \
  --i-table 10251table.qza \
  --p-sampling-depth 20102 \
  --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --output-dir all_in_one

## Faith Phylogenetic Diversity

In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file 10251_prep_690_20201201-080233.txt \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

phylogeny-informed alpha diversity using the Faith's PD metric

In [ ]:
!qiime diversity alpha-phylogenetic \
  --i-table rarefied_table.qza \
  --i-phylogeny 102511rooted-tree.qza \
  --p-metric faith_pd \
  --o-alpha-diversity faith_pd_vector.qza

#### Evenness metrics

In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file 10251_prep_690_20201201-080233.txt \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

### Beta Diversity using unweighted UniFrac distances for expermint title (Utilities from A to E )

In [ ]:
!qiime diversity beta-group-significance   \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza   \
  --m-metadata-file 10251_prep_690_20201201-080233.txt   --m-metadata-column experiment_title   \
  --o-visualization core-metrics-results/unweighted-unifrac-expermint-title-significance.qzv   \
  --p-pairwise

#### Alpha group significance

In [ ]:
%%bash
for metric in observed_features shannon pielou_e faith_pd
do
  qiime diversity alpha-group-significance \
    --i-alpha-diversity ${metric}_vector.qza \
    --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
    --o-visualization ${metric}_group_significance.qzv
done

In [ ]:
%%bash
for metric in observed_features shannon pielou_e
do
  qiime diversity alpha \
    --i-table rarefied_table.qza \
    --p-metric $metric \
    --o-alpha-diversity ${metric}_vector.qza
done

#### Alpha rarefaction plotting

In [ ]:
!qiime diversity alpha-rarefaction \
  --i-table 10251table.qza \
  --i-phylogeny 102511rooted-tree.qza \
  --p-max-depth 100000 \
  --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --o-visualization alpha-rarefaction.qzv

#### Beta Diversity

In [ ]:
%%bash
for metric in jaccard braycurtis
do
  qiime diversity beta \
    --i-table rarefied_table.qza \
    --p-metric $metric \
    --o-distance-matrix ${metric}_distance_matrix.qza
done

In [ ]:
%%bash
for metric in unweighted_unifrac weighted_unifrac
do
  qiime diversity beta-phylogenetic \
    --i-table rarefied_table.qza \
    --i-phylogeny 102511rooted-tree.qza \
    --p-metric $metric \
    --o-distance-matrix ${metric}_distance_matrix.qza
done

#### PCoA

In [ ]:
%%bash
for metric in jaccard braycurtis unweighted_unifrac weighted_unifrac
do
  qiime diversity pcoa \
    --i-distance-matrix ${metric}_distance_matrix.qza \
    --o-pcoa ${metric}_pcoa_results.qza
done

#### EMPeror

In [ ]:
%%bash
for metric in jaccard braycurtis unweighted_unifrac weighted_unifrac
do
  qiime emperor plot \
    --i-pcoa ${metric}_pcoa_results.qza \
    --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
    --o-visualization ${metric}_emperor.qzv
done

#### Time series

In [ ]:
!qiime emperor plot \
  --i-pcoa unweighted_unifrac_pcoa_results.qza \
  --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --p-custom-axes run_date \
  --o-visualization unweighted_unifrac_emperor_time.qzv

Did not work in our stuy case



#### Beta rarefaction


In [ ]:
!qiime diversity beta-rarefaction \
  --i-table 10251table.qza \
  --i-phylogeny 102511rooted-tree.qza\
  --p-metric unweighted_unifrac \
  --p-clustering-method nj \
  --p-sampling-depth 1103 \
  --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --o-visualization unweighted_unifrac_rarefaction.qzv

#### PCoA biplot

In [ ]:
!qiime feature-table relative-frequency \
  --i-table rarefied_table.qza \
  --o-relative-frequency-table relative_table.qza

In [ ]:
!qiime diversity pcoa-biplot \
  --i-pcoa unweighted_unifrac_pcoa_results.qza \
  --i-features relative_table.qza \
  --o-biplot unweighted_unifrac_pcoa_biplot.qza

In [ ]:
!qiime emperor biplot \
  --i-biplot unweighted_unifrac_pcoa_biplot.qza \
  --m-sample-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --m-feature-metadata-file taxonomy.qza \
  --o-visualization unweighted_unifrac_emperor_biplot.qzv

#### Mantel test¶


In [ ]:
!qiime diversity mantel \
  --i-dm1 braycurtis_distance_matrix.qza \
  --i-dm2 weighted_unifrac_distance_matrix.qza \
  --o-visualization bc_vs_wu_mantel.qzv

#### Procrustes analysis¶


In [ ]:
!qiime diversity procrustes-analysis \
  --i-reference braycurtis_pcoa_results.qza \
  --i-other weighted_unifrac_pcoa_results.qza \
  --o-transformed-reference braycurtis_pcoa_transformed.qza \
  --o-transformed-other weighted_unifrac_pcoa_transformed.qza

In [ ]:
!qiime emperor procrustes-plot \
  --i-reference-pcoa braycurtis_pcoa_transformed.qza \
  --i-other-pcoa weighted_unifrac_pcoa_transformed.qza \
  --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --o-visualization bc_vs_wu_procrustes.qzv

#### Group significance

In [ ]:
%%bash
for test in anosim permanova permdisp
do
  qiime diversity beta-group-significance \
    --i-distance-matrix unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
    --m-metadata-column "experiment_title" \
    --p-method $test \
    --o-visualization unweighted_unifrac_body_site_${test}.qzv
done

#### Adonis test

In [ ]:
!cat 10251/10251_prep_690_20201201-080233.txt | tr -d '-' > 10251/10251_prep_690_20201201-080233_rev.txt


In [ ]:
!qiime diversity adonis \
  --i-distance-matrix unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file 10251/10251_prep_690_20201201-080233_rev.txt \
  --p-formula "experiment_title" \
  --o-visualization unweighted_unifrac_body_site_adonis.qzv

### Extracting data from QIIME 2 results

In [ ]:
import pandas as pd

In [ ]:
from qiime2 import Metadata, Artifact

In [ ]:
metadata = Metadata.load('10251/10251_prep_690_20201201-080233.txt')
table = Artifact.load('10251table.qza')
tree = Artifact.load('102511rooted-tree.qza')
taxonomy = Artifact.load('taxonomy.qza')


## Supervised learning

In [ ]:
from qiime2.plugins.sample_classifier.pipelines import classify_samples

In [ ]:
%%capture --no-stdout
model = classify_samples(table, metadata.get_column('experiment_title'),
                         optimize_feature_selection=True,
                         parameter_tuning=True,
                         estimator='RandomForestClassifier',
                         n_estimators=20,
                         random_state=42)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)

In [ ]:
model

## Prediction results

In [ ]:
model.predictions.view(pd.Series)

In [ ]:
model.probabilities.view(pd.DataFrame)

In [ ]:
model.accuracy_results

In [ ]:
arcdir = model.accuracy_results._archiver.data_dir

In [ ]:
df = pd.read_csv(f'{arcdir}/predictive_accuracy.tsv', sep='\t', index_col=0)
df

In [ ]:
confusion = df.iloc[:-3, :-1]
confusion

In [ ]:
fig = plt.figure()
plt.imshow(confusion, cmap='Reds')
plt.xticks(range(confusion.shape[1]), confusion.columns)
plt.xlabel('Predicted')
plt.yticks(range(confusion.shape[0]), confusion.index)
plt.ylabel('Actual')
plt.colorbar()
plt.title('Confusion matrix');

In [ ]:
fig 

In [ ]:
from IPython.display import Image

In [ ]:
Image(filename=f'{arcdir}/roc_plot.png')

In [ ]:
importance = model.feature_importance.view(pd.DataFrame).squeeze()
importance

In [ ]:
taxa = taxonomy.view(pd.DataFrame)
taxa.head()

In [ ]:
taxa.loc['TACGAAGGGGGCTAGCGTTGTTCGGAATCACTGGGCGTAAAGCGCACGTAGGCGGATTTGTTAGTCAGGGGTGAAATCCCGGGGCTCAACCTCGGAACTGCCTCTGATACTGCAGATCTCGAGTCCGATAGAGGTGAGTGGAATTCCTAG']

In [ ]:
model.heatmap

In [ ]:
model.model_summary

## visualization

In [ ]:
from qiime2 import Visualization


In [ ]:
df = meta.to_dataframe()
df.head()

In [ ]:
table_qzv = Visualization.load('10251table.qzv')

In [ ]:
barplot = Visualization.load('barplot.qzv')

In [ ]:
barplot

In [ ]:
table_qzv

In [ ]:
barplotgrouped = Visualization.load('barplotgrouped.qzv')

In [ ]:
barplotgrouped

In [ ]:
repseq = Visualization.load('10251rep-seqs.qzv')

In [ ]:
repseq

#### Faith Phylogenetic Diversity

In [ ]:
faith_pd_group_significance = Visualization.load('core-metrics-results/faith-pd-group-significance.qzv')

In [ ]:
faith_pd_group_significance

#### Rarefaction Visualization

In [ ]:
rarefied_table = Visualization.load('rarefied_table.qzv')

In [ ]:
rarefied_table

#### Alpha and beta diversity analysis Visualization

In [ ]:
unweighted_unifrac_emperor = Visualization.load('all_in_one/unweighted_unifrac_emperor.qzv')

In [ ]:
unweighted_unifrac_emperor

In [ ]:
weighted_unifrac_emperor = Visualization.load('all_in_one/weighted_unifrac_emperor.qzv')

In [ ]:
weighted_unifrac_emperor

In [ ]:
jaccard_emperor = Visualization.load('all_in_one/jaccard_emperor.qzv')

In [ ]:
jaccard_emperor

In [ ]:
bray_curtis_emperor = Visualization.load('all_in_one/bray_curtis_emperor.qzv')

In [ ]:
bray_curtis_emperor

#### Evenness metrics Visualization

In [ ]:
evenness_group_significance = Visualization.load('core-metrics-results/evenness-group-significance.qzv')

In [ ]:
evenness_group_significance

#### Beta Diversity: unweighted UniFrac distances Visualization

In [ ]:
unweighted_unifrac_expermint_title_significance = Visualization.load('core-metrics-results/unweighted-unifrac-expermint-title-significance.qzv')

In [ ]:
unweighted_unifrac_expermint_title_significance

#### Alpha group significance Visualization
 

In [ ]:
observed_features_group_significance = Visualization.load('observed_features_group_significance.qzv')

In [ ]:
observed_features_group_significance

In [ ]:
shannon_group_significance = Visualization.load('shannon_group_significance.qzv')

In [ ]:
shannon_group_significance

In [ ]:
pielou_e_group_significance = Visualization.load('pielou_e_group_significance.qzv')

In [ ]:
pielou_e_group_significance

In [ ]:
faith_pd_group_significance = Visualization.load('faith_pd_group_significance.qzv')

In [ ]:
faith_pd_group_significance

#### Alpha rarefaction plotting Visualization

In [ ]:
alpha_rarefaction = Visualization.load('alpha-rarefaction.qzv')

In [ ]:
alpha_rarefaction

#### PCoA EMPeror Visualization
 

In [ ]:
jaccard_emperor = Visualization.load('jaccard_emperor.qzv')

In [ ]:
jaccard_emperor

In [ ]:
braycurtis_emperor = Visualization.load('braycurtis_emperor.qzv')

In [ ]:
braycurtis_emperor

In [ ]:
unweighted_unifrac_emperor = Visualization.load('unweighted_unifrac_emperor.qzv')

In [ ]:
unweighted_unifrac_emperor

In [ ]:
weighted_unifrac_emperor = Visualization.load('weighted_unifrac_emperor.qzv')

In [ ]:
weighted_unifrac_emperor

#### Beta rarefaction Visualization

In [ ]:
unweighted_unifrac_rarefaction = Visualization.load('unweighted_unifrac_rarefaction.qzv')

In [ ]:
unweighted_unifrac_rarefaction

#### PCoA biplot


In [ ]:
unweighted_unifrac_emperor_biplot = Visualization.load('unweighted_unifrac_emperor_biplot.qzv')

In [ ]:
unweighted_unifrac_emperor_biplot

#### Mantel test

In [ ]:
bc_vs_wu_mantel = Visualization.load('bc_vs_wu_mantel.qzv')

In [ ]:
bc_vs_wu_mantel

#### Procrustes analysis

In [ ]:
bc_vs_wu_procrustes = Visualization.load('bc_vs_wu_procrustes.qzv')

In [ ]:
bc_vs_wu_procrustes 